In [ ]:
!pip install numba

In [ ]:
from numba import cuda, int32
import numpy as np
import time

# Kernel de redução serial para a soma
@cuda.jit
def reduce_serial_kernel(d_input, d_output):
    tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    temp_sum = 0

    # Cada thread faz a acumulação de seus elementos
    for i in range(tid, d_input.size, stride):
        temp_sum += d_input[i]
    
    # Adiciona o resultado ao acumulador global usando atomicAdd para evitar condições de corrida
    cuda.atomic.add(d_output, 0, temp_sum)

# Função para executar o reduce serial e medir o tempo
def run_reduce_serial(size):
    # Inicializar o array de entrada no host
    h_input = np.ones(size, dtype=np.int32)  # Exemplo: preenche com 1 para facilitar a soma
    h_output = np.array([0], dtype=np.int32)

    # Para arrays muito pequenos, use a CPU diretamente
    if size <= 1000:
        start = time.time()
        result = np.sum(h_input)
        end = time.time()
        elapsed_time = (end - start) * 1000  # Tempo em milissegundos
        print(f"Array Size: {size}, Result: {result}, Time (CPU): {elapsed_time:.4f} ms")
        return

    # Alocar memória no dispositivo
    d_input = cuda.to_device(h_input)
    d_output = cuda.to_device(h_output)

    # Configuração do kernel
    threads_per_block = 512  # Aumentando para melhorar a ocupação
    blocks_per_grid = min(1024, (size + threads_per_block - 1) // threads_per_block)

    # Medir o tempo de execução
    start = time.time()
    
    # Executar o kernel
    reduce_serial_kernel[blocks_per_grid, threads_per_block](d_input, d_output)
    
    # Sincronizar o dispositivo para garantir que o kernel terminou
    cuda.synchronize()
    
    end = time.time()
    elapsed_time = (end - start) * 1000  # Tempo em milissegundos

    # Copiar o resultado de volta para o host
    h_output = d_output.copy_to_host()

    # Exibir o resultado e o tempo de execução
    print(f"Array Size: {size}, Result: {h_output[0]}, Time (GPU): {elapsed_time:.4f} ms")

# Tamanhos do array para o teste
array_sizes = [100, 1000, 10000, 100000, 1000000, 10000000]

# Executar a redução para cada tamanho de array
for size in array_sizes:
    run_reduce_serial(size)
